# PromoterMotifFinder

This program identifies the most common DNA sequence motifs within the promoter regions of a given list of genes. It extracts upstream promoter sequences (typically a fixed length upstream of the transcription start site) from a reference genome and search motifs (with JASPAR database)

Each motif is then counted based on the number of occurrences across all input gene regions. Finally, the motifs are ranked and listed according to their frequency, reflecting how often each motif appears within the gene set.

Similar to FIMO, but easy to use.

## Further reading
 - Known Motif Scanning: Begin by scanning your defined promoter regions for known TFBSs using established databases like JASPAR and [TRANSFAC](https://genexplain.com/transfac-product/) (via tools like [FIMO](https://meme-suite.org/meme/tools/fimo) or TRANSFAC Match). This provides immediate hypotheses about potential regulatory TFs.
 - Motif Enrichment Analysis: Follow up by performing motif enrichment analysis (e.g., with TRANSFAC FMatch or [HOMER](http://homer.ucsd.edu/homer/homer2.html)) to statistically identify which of these known motifs are significantly overrepresented in your gene set compared to the background. This pinpoints the most probable common regulating TFs.


  

In [ ]:
#@title Set parameters and `Runtime` -> `Run all`
from google.colab import files
import os
import re
import hashlib
import random
import glob

from sys import version_info
python_version = f"{version_info.major}.{version_info.minor}"

def add_hash(x,y):
  return x+"_"+hashlib.sha1(y.encode()).hexdigest()[:5]

SPECIES = "Killifish" #@param {type:"string"} ["C.elegans", "Killifish"]
#@markdown - Promoter size to search.
PROMOTER_LENGTH = 2000 #@param {type:"raw"}
#@markdown - motif match threshold.
PWM_SCORE_THRESHOLD = 0.9 # @param {type:"slider", min:0.7, max:1, step:0.1}
#@markdown - List of interested genes (WBid/ENSEMBLGENE). must be this format "["Gene1", "Gene2"...]"
GENES_OF_INTEREST = ["ENSNFUG00015001047", "ENSNFUG00015001509", "ENSNFUG00015003068", "ENSNFUG00015006014", "ENSNFUG00015006496", "ENSNFUG00015007030", "ENSNFUG00015008382", "ENSNFUG00015009619", "ENSNFUG00015010807", "ENSNFUG00015011287", "ENSNFUG00015011653", "ENSNFUG00015012125", "ENSNFUG00015012302", "ENSNFUG00015012412", "ENSNFUG00015017059", "ENSNFUG00015018034", "ENSNFUG00015019527", "ENSNFUG00015020785", "ENSNFUG00015020981", "ENSNFUG00015022868", "ENSNFUG00015023747", "ENSNFUG00015024112", "ENSNFUG00015024297", "ENSNFUG00015025252", "ENSNFUG00015025346", "ENSNFUG00015020771", "ENSNFUG00015012024"] #@param {type:"raw"}

jaspar_taxid = {"C.elegans": 6239,
                "Killifish": 8078,
                "Mouse": 10090}

jaspar_taxgrp = {"C.elegans": "Nematodes",
                "Killifish": "Vertebrates",
                "Mouse": "Vertebrates",

}
genome_wget_addr = {"C.elegans": "1H6cAsIMS2wH2Wh8bii8N4n9sVie-unFv",
                    "Killifish": "1lEMBw8I_41gplugWgJh-9uS9_BDCVtMW",
                    "Mouse": "1mAjaSJG21jKFlQBt2qYlDKLDTTrBE6I6",
                    }
gtf_wget_addr = {"C.elegans": "1hitCMc-VVyscVvkfe0QCDz5nKTGHyxnA",
                "Killifish": "16TbToIZQFmcTxsgP__ZBr3fdykG5CJl0",
                 "Mouse": "1fzKgOW6fHWb2lY-OOSZ0k-r1fnsGCpqd",
                 }
jaspar_wget_addr = {"C.elegans": "1d0Po7ed-Eq6Mm9NLu4paRGH4ZMK0WlQU",
                "Killifish": "1OEy3MePEhZ9DtdyzuFwtFQo5G11bVRsc",
                 "Mouse": "1OEy3MePEhZ9DtdyzuFwtFQo5G11bVRsc",
                }

background_files = {
    "C.elegans": "1D0hb2uYuK9kepcQfTriGeSLQoJT6LyOc",
    "Killifish": "14gGqQfZpEoPVT1BX9xLFz2OL9FE1WvjT",
}

target_taxid = jaspar_taxgrp[SPECIES]
GENOME_WGET = genome_wget_addr[SPECIES]
GTF_WGET = gtf_wget_addr[SPECIES]
JASPAR_WGET = jaspar_wget_addr[SPECIES]
BGDIST_WGET = background_files[SPECIES]

GeneIDheader = {"C.elegans": "WB",
                "Killifish": "ENSNFUG",
                "Mouse": "ENSMUSG"}

for x in GENES_OF_INTEREST:
    assert GeneIDheader[SPECIES] in x, "Check your gene list again"

# Run

## Set up

In [ ]:
pip install biopython

In [ ]:
import sys
import requests
import math
import dill, pickle, os
from collections import defaultdict, Counter
from Bio import SeqIO # You'll need Biopython installed (pip install biopython)
from Bio import motifs
from Bio.Seq import Seq
from statsmodels.stats.multitest import fdrcorrection
from scipy.stats import fisher_exact
import tqdm
import datetime
import pytz
import pandas as pd
from google.colab import files

In [ ]:
# --- Configuration ---
GENOME_FASTA_GZ = "/content/genome.fa.gz"
REFSEQ_GTF_GZ = "/content/gtf.gz"

GENOME_FASTA = "/content/genome.fa"
REFSEQ_GTF = "/content/gtf"
JASPAR_PFM_FILE = "/content/jaspar"
BG_FILE = "/content/bg_killifish.tar.gz"

!gdown $GENOME_WGET -O $GENOME_FASTA_GZ
!sleep 2
!gunzip -f $GENOME_FASTA_GZ
!sleep 2

!gdown $GTF_WGET -O $REFSEQ_GTF_GZ
!sleep 2
!gunzip -f $REFSEQ_GTF_GZ
!sleep 2

!gdown $JASPAR_WGET -O $JASPAR_PFM_FILE
!sleep 2

!gdown $BGDIST_WGET
!tar -xf $BG_FILE

## src

In [ ]:
def calculate_pseudocounts(motif):
    """Calculate pseudocounts.

    Computes the root square of the total number of sequences multiplied by
    the background nucleotide.
    """
    alphabet = motif.alphabet
    background = motif.background

    # It is possible to have unequal column sums so use the average
    # number of instances.
    total = 0
    for i in range(motif.length):
        total += sum(motif.counts[letter][i] for letter in alphabet)

    avg_nb_instances = total / motif.length
    sq_nb_instances = math.sqrt(avg_nb_instances)

    if background:
        background = dict(background)
    else:
        background = dict.fromkeys(sorted(alphabet), 1.0)

    total = sum(background.values())
    pseudocounts = {}

    for letter in alphabet:
        background[letter] /= total
        pseudocounts[letter] = sq_nb_instances * background[letter]

    return pseudocounts


# --- 1. Load Genome ---
def load_genome(fasta_file):
    """
    Loads the genome sequence from a FASTA file into a dictionary.
    Keys are chromosome names, values are SeqIO objects.
    """
    print(f"Loading genome from {fasta_file}...")
    genome = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))
    print("Genome loaded.")
    return genome


# --- 2. Parse Gene Annotations and Get Promoter Coordinates ---
def get_promoter_coordinates(gtf_file, genes_of_interest, promoter_length):
    """
    Parses a GTF/GFF file to get gene coordinates and calculate promoter regions.
    Returns a dictionary: {gene_id: {'chr': str, 'start': int, 'end': int, 'strand': str}}
    Note: This is a simplified parser. Real GTF/GFF parsing can be complex.
    """
    print(f"Parsing annotations from {gtf_file} and identifying promoter regions...")
    gene_promoters = {}
    with open(gtf_file, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue
            parts = line.strip().split('\t')
            # Assuming GTF/GFF format: seqname source feature start end score strand frame attributes
            seqname = parts[0]
            feature_type = parts[2]
            start = int(parts[3])
            end = int(parts[4])
            strand = parts[6]
            attributes = parts[8]

            if feature_type == "gene": # Or "CDS" depending on what you define as gene start
                # Extract gene ID from attributes (this part is highly dependent on GTF/GFF format)
                # Example for Ensembl GTF: gene_id "WBGene0000001"; gene_version "1";
                #gene_id_match = [attr.strip().split(' ')[1].strip('"') for attr in attributes.split(';') if "gene_id" in attr]
                gene_id_match = attributes.split(';')[0].replace('gene_id ','').replace('"','')
                if not gene_id_match:
                    continue
                gene_id = gene_id_match

                if genes_of_interest and gene_id not in genes_of_interest:
                    continue

                promoter_start = -1
                promoter_end = -1

                if strand == '+':
                    # Promoter is upstream of the start site
                    promoter_start = start - promoter_length
                    promoter_end = start - 1 # Or just 'start' if you want to include the TSS base
                elif strand == '-':
                    # Promoter is upstream of the end site (for reverse strand)
                    promoter_start = end + 1 # Or just 'end'
                    promoter_end = end + promoter_length

                # Ensure promoter coordinates are not negative
                promoter_start = max(0, promoter_start)

                if promoter_start != -1 and promoter_end != -1:
                    gene_promoters[gene_id] = {
                        'chr': seqname,
                        'start': promoter_start,
                        'end': promoter_end,
                        'strand': strand
                    }
    print(f"Found promoter regions for {len(gene_promoters)} genes.")
    return gene_promoters


# --- 3. Extract Promoter Sequences ---
def extract_promoter_sequences(genome, gene_promoters):
    """
    Extracts the DNA sequence for each promoter region.
    Returns a dictionary: {gene_id: 'sequence'}
    """
    print("Extracting promoter sequences...")
    promoter_sequences = {}
    for gene_id, coords in gene_promoters.items():
        chrom = coords['chr']
        start = coords['start']
        end = coords['end']
        strand = coords['strand']

        if chrom not in genome:
            print(f"Warning: Chromosome '{chrom}' not found in genome for gene {gene_id}. Skipping.")
            continue

        # Biopython Seq objects are 0-indexed, slice is [start:end]
        # Adjusting for 1-based GTF to 0-based Python slicing
        seq_obj = genome[chrom].seq[start:end]

        if strand == '-':
            seq_obj = seq_obj.reverse_complement()
        promoter_sequences[gene_id] = str(seq_obj).upper() # Convert to string and uppercase
    print(f"Extracted sequences for {len(promoter_sequences)} promoters.")
    return promoter_sequences


# --- Revised Common K-mer to Known Motif Comparison (using PFM/PWM scanning) ---
def compare_common_kmers_with_pfms(common_kmers, jaspar_motifs, promoter_sequences, score_threshold):
    """
    Compares found common k-mers by scanning promoter regions with JASPAR PWMs.
    This is a more robust way to link found k-mers to known motifs.
    It identifies which promoter regions contain matches to known motifs.
    """
    print(f"\n--- Scanning promoter regions for known JASPAR motifs (score threshold: {score_threshold*100:.1f}%) ---")
    motif_hits_in_promoters = defaultdict(lambda: defaultdict(list)) # {motif_name: {gene_id: [(pos, score, seq, strand), ...]}}
    overall_motif_counts = defaultdict(int) # {motif_name: total occurrences}

    for motif_name, motif_obj in jaspar_motifs.items():
        print(f"Scanning for motif: {motif_name} (Length: {len(motif_obj)})")
        for gene_id, promoter_seq in promoter_sequences.items():
            # Ensure promoter sequence is long enough to contain the motif
            if len(promoter_seq) >= len(motif_obj):
                matches = scan_sequence_with_pwm(promoter_seq, motif_obj, score_threshold)
                if matches:
                    motif_hits_in_promoters[motif_name][gene_id].extend(matches)
                    overall_motif_counts[motif_name] += len(matches)

    print("\n--- Summary of Known Motif Hits in Promoters ---")
    if overall_motif_counts:
        for motif_name, count in sorted(overall_motif_counts.items(), key=lambda item: item[1], reverse=True):
            continue#print(f"Motif: {motif_name} (Total hits: {count})")
            # You can uncomment to print individual hits:
            # for gene_id, hits in motif_hits_in_promoters[motif_name].items():
            #     for pos, score, seq, strand in hits:
            #         print(f"  Gene {gene_id}: Match '{seq}' at position {pos} (strand {strand}, score {score:.2f})")
    else:
        print("No known motifs found in promoter regions with the given threshold.")

    # You could also link your *found k-mers* directly to the motifs
    # by checking if your exact k-mers (from common_kmers) achieve a high score
    # against the PWMs, but the primary use of PWMs is to scan a sequence.
    # For now, we'll just report the found motif occurrences.
    return motif_hits_in_promoters


# --- scan_sequence_with_pwm (The latest corrected version with .min_score()/.max_score())
def scan_sequence_with_pwm(sequence, motif_obj, threshold_percent):
    """
    Scans a DNA sequence for matches to a motif using its PWM/PSSM.
    Returns a list of (start_position, score, matching_sequence) tuples
    for matches exceeding the threshold, considering both strands.
    """
    matches = []

    pssm = motif_obj.pssm

    min_score = pssm.min
    max_score = pssm.max

    if (max_score - min_score) > 0:
        score_cutoff = (max_score - min_score) * threshold_percent + min_score
    else:
        # print(f"Warning: Motif '{motif_obj.name}' has min_score == max_score. Skipping scanning.", file=sys.stderr)
        return [] # Return empty if no valid score range

    # Scan forward strand
    for position, score in pssm.search(str(sequence), threshold=score_cutoff):
        if position + len(motif_obj) <= len(sequence):
            matched_seq = sequence[position : position + len(motif_obj)]
            matches.append((position, score, matched_seq, '+'))

    # Scan reverse complement strand
    rev_comp_sequence_obj = Seq(sequence).reverse_complement()
    rev_comp_sequence_str = str(rev_comp_sequence_obj)

    for position, score in pssm.search(rev_comp_sequence_str, threshold=score_cutoff):
        if position + len(motif_obj) <= len(rev_comp_sequence_str):
            matched_seq = rev_comp_sequence_str[position : position + len(motif_obj)]
            matches.append((position, score, matched_seq, '-'))

    return matches


def scan_for_hits(promoter_seqs, jaspar_motifs):
    """Helper function to scan a set of promoters and return motif hits."""
    motif_hits = defaultdict(lambda: {'count': 0, 'genes': set()})
    for motif_name, motif_obj in tqdm.tqdm(jaspar_motifs.items()):
        if not motif_obj or len(motif_obj) == 0:
            continue
        try:
            _ = motif_obj.pssm
        except Exception as e:
            continue # Already warned in parsing step

        for gene_id, promoter_seq in promoter_seqs.items():
            if len(promoter_seq) >= len(motif_obj):
                matches = scan_sequence_with_pwm(promoter_seq, motif_obj, PWM_SCORE_THRESHOLD)
                if matches:
                    motif_hits[motif_name]['count'] += len(matches)
                    motif_hits[motif_name]['genes'].add(gene_id)
    return motif_hits


def load_jaspar_pfm(file_path):
    """
    Parses a JASPAR-like PFM file with a custom floating-point format.
    Assumes 4 rows (A, C, G, T) for each motif.
    Converts float counts to integers by rounding for Bio.motifs.Motif object.
    Crucially, passes PSSM_PSEUDOCOUNTS to the Motif constructor.
    """
    print(f"Starting custom parsing of PFM file: {file_path}")
    jaspar_motifs = {}

    current_motif_name = None
    current_pfm_rows = [] # Will temporarily store [A_counts, C_counts, G_counts, T_counts]

    try:
        with open(file_path, "r") as f:
            for line_num, line in enumerate(f, 1):
                original_line = line # Keep original for error messages
                line = line.strip()

                if not line: # Skip empty lines
                    continue

                if line.startswith('>'):
                    # A new motif is starting. First, try to process the previous one.
                    if current_motif_name: # Check if there was a previous motif being built
                        if len(current_pfm_rows) == 4:
                            try:
                                integer_pfm_dict = {
                                    'A': [int(round(x)) for x in current_pfm_rows[0]],
                                    'C': [int(round(x)) for x in current_pfm_rows[1]],
                                    'G': [int(round(x)) for x in current_pfm_rows[2]],
                                    'T': [int(round(x)) for x in current_pfm_rows[3]],
                                }

                                # Check if any row is empty (e.g., due to parsing errors resulting in empty lists)
                                if any(not row for row in integer_pfm_dict.values()):
                                    raise ValueError("One or more PFM rows are empty after conversion.")

                                # THE FIX HERE: Pass pseudocounts to the Motif constructor
                                motif_obj = motifs.Motif(alphabet="ACGT", counts=integer_pfm_dict)
                                motif_obj.pseudocounts = calculate_pseudocounts(motif_obj)

                                motif_obj.name = current_motif_name
                                jaspar_motifs[current_motif_name] = motif_obj
                                # print(f"Successfully parsed motif: {current_motif_name}") # Debugging line
                            except Exception as e:
                                print(f"Error processing motif '{current_motif_name}' (started around line {line_num - len(current_pfm_rows) -1}): Incomplete or malformed data preventing motif creation: {e}", file=sys.stderr)
                        else:
                            print(f"Warning: Motif '{current_motif_name}' (started around line {line_num - len(current_pfm_rows) -1}) has {len(current_pfm_rows)} data rows instead of 4. Skipping.", file=sys.stderr)

                    # Reset for the new motif
                    parts = line[1:].split(' ')
                    if len(parts) > 1:
                        current_motif_name = parts[1]
                    else:
                        current_motif_name = parts[0]
                    current_pfm_rows = [] # Clear rows for the new motif
                    # print(f"Detected new motif header: {current_motif_name}") # Debugging line

                else:
                    # This is a data line (A, C, G, T counts)
                    if current_motif_name is None:
                        print(f"Warning: Data line found before any motif header at line {line_num}: '{original_line.strip()}'. Skipping.", file=sys.stderr)
                        continue

                    if len(current_pfm_rows) >= 4:
                        print(f"Warning: Motif '{current_motif_name}' at line {line_num}: More than 4 data rows detected. Skipping extra row: '{original_line.strip()}'.", file=sys.stderr)
                        continue

                    try:
                        counts = [float(x) for x in line.split()]
                        # Essential check: ensure 'counts' list is not empty after splitting
                        if not counts:
                            print(f"Warning: Motif '{current_motif_name}' at line {line_num}: Data line is empty after splitting. Skipping: '{original_line.strip()}'.", file=sys.stderr)
                            continue

                        current_pfm_rows.append(counts)
                    except ValueError as e:
                        print(f"Error parsing data line for motif '{current_motif_name}' at line {line_num}: '{original_line.strip()}'. Error: {e}. Skipping this line.", file=sys.stderr)

        # After the loop, process the very last motif in the file
        if current_motif_name: # Check if there was any motif at all
            if len(current_pfm_rows) == 4:
                try:
                    integer_pfm_dict = {
                        'A': [int(round(x)) for x in current_pfm_rows[0]],
                        'C': [int(round(x)) for x in current_pfm_rows[1]],
                        'G': [int(round(x)) for x in current_pfm_rows[2]],
                        'T': [int(round(x)) for x in current_pfm_rows[3]],
                    }
                    if any(not row for row in integer_pfm_dict.values()):
                        raise ValueError("One or more PFM rows are empty after conversion in final motif.")

                    # THE FIX HERE: Pass pseudocounts to the Motif constructor for the last motif
                    motif_obj = motifs.Motif(alphabet="ACGT", counts=integer_pfm_dict)
                    motif_obj.pseudocounts = calculate_pseudocounts(motif_obj)

                    motif_obj.name = current_motif_name
                    jaspar_motifs[current_motif_name] = motif_obj
                    # print(f"Successfully parsed final motif: {current_motif_name}") # Debugging line
                except Exception as e:
                    print(f"Error processing final motif '{current_motif_name}': Incomplete or malformed data preventing motif creation: {e}", file=sys.stderr)
            else:
                print(f"Warning: Final motif '{current_motif_name}' has {len(current_pfm_rows)} data rows instead of 4. Skipping.", file=sys.stderr)

    except FileNotFoundError:
        print(f"Critical Error: PFM file not found at {file_path}", file=sys.stderr)
        return {} # Return empty dict on critical file error
    except Exception as e:
        print(f"An unexpected critical error occurred during file reading: {e}", file=sys.stderr)
        return {} # Return empty dict on critical file error

    print(f"Finished custom PFM parsing. Loaded {len(jaspar_motifs)} motifs.")
    return jaspar_motifs

## main

In [ ]:
# --- 1. Load Genome ---
genome_data = load_genome(GENOME_FASTA)
if not genome_data:
    print("Error: Genome data could not be loaded. Exiting.")
    sys.exit(1)

# --- 2. Load JASPAR Motifs ---
jaspar_motifs_obj = load_jaspar_pfm(JASPAR_PFM_FILE)
if not jaspar_motifs_obj:
    print("Error: No JASPAR motifs loaded from the PFM file. Check file path and format. Exiting.")
    sys.exit(1)

# --- 3. Get Promoter Coordinates for TARGET Genes ---
print(f"\n--- Processing Target Genes: {len(GENES_OF_INTEREST)} genes specified ---")
target_promoter_coords = get_promoter_coordinates(REFSEQ_GTF, GENES_OF_INTEREST, PROMOTER_LENGTH)

if not target_promoter_coords:
    print("Warning: No promoter coordinates found for target genes. Cannot scan for motifs. Exiting.")
    sys.exit(1)

genes_to_process_target = {gene_id: coords for gene_id, coords in target_promoter_coords.items()
                            if coords['chr'] in genome_data}

if not genes_to_process_target:
    print("Warning: No target genes to process after filtering by genome presence. Exiting.")
    sys.exit(1)

# --- 4. Extract Promoter Sequences for TARGET Genes ---
target_promoter_seqs = extract_promoter_sequences(genome_data, genes_to_process_target)
if not target_promoter_seqs:
    print("Warning: No promoter sequences could be extracted for target genes. Exiting.")
    sys.exit(1)

# --- 5. Scan Target Promoter Regions for Known Motifs ---
print("\n--- Scanning Target Promoter Regions for Known Motifs ---")
target_motif_hits = defaultdict(lambda: {'count': 0, 'genes': set()}) # {motif_name: {'count': N, 'genes': {gene_ids}}}

for motif_name, motif_obj in jaspar_motifs_obj.items():
    if not motif_obj or len(motif_obj) == 0:
        print(motif_obj)
        continue # Skip invalid motifs

    try:
        _ = motif_obj.pssm # Test PSSM access
    except Exception as e:
        print(f"Warning: Could not generate PSSM for motif '{motif_name}': {e}. Skipping.", file=sys.stderr)
        continue

    # print(f"Scanning for motif: {motif_name} (Length: {len(motif_obj)}) in target promoters...")
    for gene_id, promoter_seq in target_promoter_seqs.items():
        if len(promoter_seq) >= len(motif_obj):
            matches = scan_sequence_with_pwm(promoter_seq, motif_obj, PWM_SCORE_THRESHOLD)
            if matches:
                target_motif_hits[motif_name]['count'] += len(matches)
                target_motif_hits[motif_name]['genes'].add(gene_id)
        # else:
        #     print(f"Debug: Promoter for {gene_id} too short for motif {motif_name}. Skipping.")

print("\n--- Motif Scan Results in Target Promoters ---")
if target_motif_hits:
    sorted_target_hits = sorted(target_motif_hits.items(), key=lambda item: item[1]['count'], reverse=True)
else:
    print("No motifs found in target promoter regions with the given threshold.")

final = []
for k, v in sorted_target_hits:
    final.append({'motif': k, 'hits': v['count'], 'genes': len(v['genes']), 'gene list': v['genes']})

genelist_final = pd.DataFrame().from_dict(final)

target_background_path = f"background_{SPECIES}_PWM_{PWM_SCORE_THRESHOLD}.pkl"
if not os.path.exists(target_background_path):
    current_date = datetime.datetime.now().astimezone(pytz.timezone('CET'))
    datestamp = f"{str(current_date.year)[-2:]}{current_date.month:02d}{current_date.day:02d}_{current_date.strftime('%X').replace(':','')}"
    download_name = f"motif_search_{datestamp}.csv"
    genelist_final.to_csv(download_name)
    files.download(f"{download_name}")
    raise('no stat test')

In [ ]:
# --- Step 6: Perform Statistical Enrichment Analysis ---
with open(target_background_path, 'rb') as outp:
    background_motif_hits = dill.load(outp, pickle.HIGHEST_PROTOCOL)

all_gene_promoter_coords = get_promoter_coordinates(REFSEQ_GTF, [], PROMOTER_LENGTH)

valid_promoter_coords = {gene_id: coords for gene_id, coords in all_gene_promoter_coords.items()
                            if coords['chr'] in genome_data}

total_target_genes = len(GENES_OF_INTEREST)
total_background_genes = len(valid_promoter_coords)

enrichment_results = []

for motif_name, motif_data in jaspar_motifs_obj.items():
    if motif_name not in target_motif_hits and motif_name not in background_motif_hits:
        continue

    count_target_has_motif = len(target_motif_hits.get(motif_name, {}).get('genes', set()))
    count_background_has_motif = len(background_motif_hits.get(motif_name, {}).get('genes', set()))

    count_target_no_motif = total_target_genes - count_target_has_motif
    count_background_no_motif = total_background_genes - count_background_has_motif

    table = [[count_target_has_motif, count_background_has_motif],
              [count_target_no_motif, count_background_no_motif]]

    if any(sum(row) == 0 for row in table) or any(sum(col) == 0 for col in zip(*table)):
        p_value = 1.0
        odds_ratio = float('nan')
    else:
        odds_ratio, p_value = fisher_exact(table, alternative='greater')

    enrichment_results.append({
        'motif_name': motif_name,
        'target_genes_with_motif': count_target_has_motif,
        'background_genes_with_motif': count_background_has_motif,
        'total_target_genes': total_target_genes,
        'total_background_genes': total_background_genes,
        'odds_ratio': odds_ratio,
        'p_value': p_value
    })

# Perform Multiple Testing Correction (Benjamini-Hochberg FDR)
valid_p_values = [res['p_value'] for res in enrichment_results]

if valid_p_values:
    reject, adjusted_p_values = fdrcorrection(valid_p_values, alpha=0.05, method='indep')

    p_idx = 0
    for res in enrichment_results:
        if res['p_value'] is not None:
            res['adjusted_p_value'] = adjusted_p_values[p_idx]
            res['significant_fdr'] = reject[p_idx]
            p_idx += 1
        else:
            res['adjusted_p_value'] = float('nan')
            res['significant_fdr'] = False

genelist_final.index = genelist_final['motif']
er = pd.DataFrame().from_dict(enrichment_results)
er.index = er['motif_name']

stat_final = pd.concat([genelist_final['gene list'], er], axis=1)

current_date = datetime.datetime.now().astimezone(pytz.timezone('CET'))
datestamp = f"{str(current_date.year)[-2:]}{current_date.month:02d}{current_date.day:02d}_{current_date.strftime('%X').replace(':','')}"
download_name = f"motif_search_with_fdr_{datestamp}.csv"
pd.DataFrame().from_dict(stat_final).to_csv(download_name)
files.download(f"{download_name}")

# Note
Vibe coding with Gemini & [Youngjun Park](https://github.com/iron-lion) (youngjun.park@age.mpg.de)